In [1]:
import glob
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from IPython.display import Image
import time

# importing required libraries
import pandas as pd
import matplotlib.patches as patches
import matplotlib.pyplot as plt
import pandas as pd
from get_iou import get_max_iou, get_iou, get_overlap
import pickle
import random
import json
import os
from collections import Counter
import cv2
import numpy as np

from metric_tablebank import metric_table_bank_union

In [2]:
path_data = "/data/rali5/Tmp/yockelle/TableBank/TableBank/Detection/"

In [3]:
def get_test_data_latex():
    latex = pickle.load( open(path_data + "annotations/COCO_tablebank_latex_test.pkl", "rb" ) )
    return latex

def get_test_data_word():
    word = pickle.load( open(path_data + "annotations/COCO_tablebank_word_test.pkl", "rb" ) )
    return word

def get_test_data_both():
    latex = get_test_data_latex()
    word = get_test_data_word()
    total_test = latex + word
    random.shuffle(total_test)
    return total_test

def get_test_data_publaynet():
    test_publaynet = pickle.load( open("../PubLayNet/COCO_test_set_table_only.pkl", "rb" ) )
    for image in test_publaynet:
        annotation_array = []
        for annot in image["annotations"]:
            annot["category_id"] = 0
            annotation_array.append(annot)
        image['categories'] = [{'supercategory': '', 'id': 0, 'name': 'table'}]
        image["annotations"] = annotation_array
        image["file_name"] = "/data/rali5/Tmp/yockelle/PubLayNet/" + image["file_name"]
    return test_publaynet

In [9]:
test_data_latex = get_test_data_publaynet()

In [5]:
"""
test_latex_truth = {}

test_data = get_test_data_latex()

threshold = 0.975
precision_tab = []
count = 0

random_count = 0

all_image = []
all_annotations = []

for i, example in enumerate(test_data):

  take_the_image = True

  one_image = {}
  
  if i%1000 == 0:
    print(i)

  only_name = example["file_name"].split("/")[-1]
  #image_name = "/data/rali5/Tmp/yockelle/PubLayNet/" + image_dict["file_name"]

  img = cv2.imread(example["file_name"])

  for annot in example["annotations"]:
    #if scores_detectron[j]>threshold:
    p = annot["bbox"]
    x1 = p[0]
    y1 = p[1]
    w = p[2]
    h = p[3]
    x2 = p[0] + w
    y2 = p[1] + h
    if y1 == y2 or y2<y1:
      break
      #cv2.rectangle(img, (x_min, y_max), (x_max, y_min), (255,0,0), 1)
      
    one_annotations = {}
    one_annotations["id"] = count
    one_annotations["segmentation"] = []
    one_annotations["area"] = float(p[-1] * p[-2])
    one_annotations["iscrowd"] = 0
    one_annotations["ignore"] = 0
    one_annotations["image_id"] = i
    one_annotations["bbox"] = [float(pp) for pp in p]
    one_annotations["category_id"] = 0

    all_annotations.append(one_annotations)
    count += 1

  one_image["id"] = i
  one_image["file_name"] = only_name
  one_image["width"] = example["width"]
  one_image["height"] = example["height"]

  all_image.append(one_image)
  

test_latex_truth["images"] = all_image
test_latex_truth["annotations"] = all_annotations
test_latex_truth["categories"] = [{"id": 0, "name": "table", "supercategory": "none"}]

out_file = open("train_latex_test_latex_truth.json", "w")
json.dump(test_latex_truth, out_file, indent = 6)
out_file.close()
"""

'\ntest_latex_truth = {}\n\ntest_data = get_test_data_latex()\n\nthreshold = 0.975\nprecision_tab = []\ncount = 0\n\nrandom_count = 0\n\nall_image = []\nall_annotations = []\n\nfor i, example in enumerate(test_data):\n\n  take_the_image = True\n\n  one_image = {}\n  \n  if i%1000 == 0:\n    print(i)\n\n  only_name = example["file_name"].split("/")[-1]\n  #image_name = "/data/rali5/Tmp/yockelle/PubLayNet/" + image_dict["file_name"]\n\n  img = cv2.imread(example["file_name"])\n\n  for annot in example["annotations"]:\n    #if scores_detectron[j]>threshold:\n    p = annot["bbox"]\n    x1 = p[0]\n    y1 = p[1]\n    w = p[2]\n    h = p[3]\n    x2 = p[0] + w\n    y2 = p[1] + h\n    if y1 == y2 or y2<y1:\n      break\n      #cv2.rectangle(img, (x_min, y_max), (x_max, y_min), (255,0,0), 1)\n      \n    one_annotations = {}\n    one_annotations["id"] = count\n    one_annotations["segmentation"] = []\n    one_annotations["area"] = float(p[-1] * p[-2])\n    one_annotations["iscrowd"] = 0\n    one

# Predictions

In [6]:
out_file = open("train_latex_test_pln_preds_N.json", "r")
preds = json.loads(out_file.read())
out_file.close() 

In [7]:
out_file = open("train_latex_test_pln_truth_N.json", "r")
truth = json.loads(out_file.read())
out_file.close() 

In [ ]:
'/data/rali5/Tmp/yockelle/PubLayNet/data/publaynet/train/PMC4494570_00004.jpg'

In [15]:
rearranged_preds = []

for i, annot in enumerate(preds["annotations"]):

    current_id = annot["image_id"]
    image_name = [d for d in preds["images"] if d["id"] == current_id][0]['file_name']

    if i == 0:
        new_dict_one_image = {}
        new_dict_one_image["image_id"] = annot["image_id"]
        new_dict_one_image["image_name"] = "/data/rali5/Tmp/yockelle/PubLayNet/data/publaynet/train/" + image_name
        new_dict_one_image["bbox"] = [annot["bbox"]]
    else:
        if current_id == previous_id:
            #if len(new_dict_one_image["bbox"]) > 1:
            new_dict_one_image["bbox"].append(annot["bbox"])
        else:
            # if we are at a new instance, we add the dict 
            rearranged_preds.append(new_dict_one_image)
            new_dict_one_image = {}
            new_dict_one_image["image_id"] = annot["image_id"]
            new_dict_one_image["image_name"] = "/data/rali5/Tmp/yockelle/PubLayNet/data/publaynet/train/" + image_name
            new_dict_one_image["bbox"] = [annot["bbox"]]

    previous_id = current_id

In [16]:
all_images_truth = preds["images"]
sum_numerator = 0
sum_numerator_old = 0
sum_denominator_precision = 0
sum_denominator_recall = 0

for annot in rearranged_preds:
    id = annot["image_id"]
    matches = [d for d in truth["annotations"] if d["image_id"] == id]

    #matches_name = [d for d in rearranged_preds["annotations"] if d["image_id"] == id]

    bbox_ground_truth = []
    for match in matches:
        p = match["bbox"]
        x1 = p[0]
        y1 = p[1]
        w = p[2]
        h = p[3]
        x2 = x1 + w
        y2 = y1 + h
        bbox_ground_truth.append([x1,y1,x2,y2])


    bbox_preds = []
    for match in annot["bbox"]:
        p = match
        x1 = p[0]
        y1 = p[1]
        w = p[2]
        h = p[3]
        x2 = x1 + w
        y2 = y1 + h
        bbox_preds.append([x1,y1,x2,y2])
    
    result = metric_table_bank_union(np.array(bbox_ground_truth), np.array(bbox_preds))
    numerator, denominator_precision, denominator_recall, old_way_area_union = result

    sum_numerator += numerator
    sum_numerator_old += old_way_area_union
    sum_denominator_precision += denominator_precision
    sum_denominator_recall += denominator_recall


precision = sum_numerator/sum_denominator_precision
recall = sum_numerator/sum_denominator_recall
f1 = (2 * precision * recall) / (precision + recall)

print("First")
print("Precision ", precision)
print("Recall ", recall)
print("F1 ", f1)

First
Precision  0.587359651756573
Recall  0.9810548902062716
F1  0.7347956082382325
